# Dieses Notebook zeigt an einem einfachen Beispiel, wie mit der Verwaltungsschale gearbeitet werden kann.

Zuerst wird die Python Implementierung der AAS installiert.
Quelle: https://github.com/eclipse-basyx/basyx-python-sdk


In [ ]:
pip install basyx-python-sdk

Importieren Sie alle Eclipse BaSyx Python SDK Klassen aus dem model-Paket

In [29]:
from basyx.aas import model

In diesem Tutorial erhalten Sie eine schrittweise Anleitung, wie Sie eine Asset Administration Shell (AAS) und alle
erforderlichen Objekte erzeugen. Zunächst benötigen Sie ein Asset, für das Sie eine AAS erstellen möchten, dargestellt durch ein Asset-Objekt.</br>
Danach kann eine Asset Administration Shell erstellt werden, die einen Verweis auf dieses Asset enthält. Dann ist es möglich Submodels zum AAS hinzuzufügen. Die Submodels können SubmodelElements enthalten.</br>
</br>
### 1. Schritt eine einfache Verwaltungsschale erstellen:


In [30]:
# Schritt 1.1: Erstellen eines Bezeichners für das Asset
# Hier verwenden wir einen IRI-Bezeichner.
identifier = model.Identifier(id_='https://acplt.org/Simple_Asset',
                              id_type=model.IdentifierType.IRI)
 
# Schritt 1.2: ein Asset erzeugen
asset = model.Asset(
    kind=model.AssetKind.INSTANCE,  # define that the Asset is of kind instance
    identification=identifier  # set identifier
)

### 2. Schritt: Erstellen Sie eine einfache Asset-Verwaltungsschale, die einen Verweis auf das Asset enthält.

In [31]:
identifier = model.Identifier('https://acplt.org/Simple_AAS', model.IdentifierType.IRI)
aas = model.AssetAdministrationShell(
    identification=identifier,  # set identifier
    asset=model.AASReference.from_referable(asset)  # generate a Reference object to the Asset (using its identifier)
)

### 3. Schritt: Erstellen eines einfachen Submodells ohne Submodel Elemente

In [32]:
#Schritt 3.1: Erstellen des Submodel-Objekts
identifier = model.Identifier('https://acplt.org/Simple_Submodel', model.IdentifierType.IRI)
submodel = model.Submodel(
    identification=identifier
)

# Schritt 3.2: Erstellen Sie einen Verweis auf dieses Teilmodell und fügen Sie es der Gruppe "Teilmodell" 
# der Asset-Verwaltungsschale hinzu.
aas.submodel.add(model.AASReference.from_referable(submodel))

### Schritt 4: Erstellen Sie eine einfache Eigenschaft und fügen Sie sie dem Untermodell hinzu

In [33]:
# Schritt 4.1: Erstellen eines globalen Verweises auf eine semantische Beschreibung der Eigenschaft.
# Ein globaler Verweis besteht aus einem Schlüssel, der auf die Adresse verweist, 
# unter der die semantische Beschreibung gespeichert ist.
semantic_reference = model.Reference(
    (model.Key(
        type_=model.KeyElements.GLOBAL_REFERENCE,
        local=False,
        value='http://acplt.org/Properties/SimpleProperty',
        id_type=model.KeyType.IRI
    ),)
)

# Schritt 4.2: Erstellen der einfachen Eigenschaft
property_ = model.Property(
    id_short='ExampleProperty',  # Identifying string of the element within the Submodel namespace
    value_type=model.datatypes.String,  # Data type of the value
    value='exampleValue',  # Value of the Property
    semantic_id=semantic_reference  # set the semantic reference
)

# Schritt 4.3: Hinzufügen der Eigenschaft zum Teilmodell
submodel.submodel_element.add(property_)


## Serialisierung

Nun wollen wir in weiteren Schritten die eben erzeugte Verwaltungsschale serialisieren, um sie z.B. an Kunden weiterzugeben<p>
`AASToJsonEncoder` aus dem Modul `aas.adapter.json` ist eine benutzerdefinierte JSONEncoder-Klasse zur Serialisierung
von Asset Administration Shell-Daten in das offizielle JSON-Format gemäß
'Details der Asset Administration Shell', Kapitel 5.5, unter Verwendung der in Python eingebauten JSON-Bibliothek. Die Methoden `json.dump()` und `json.dumps()` sind in der Lage, AAS-Objekte innerhalb der gedumpten Datenstrukrur korrekt abzubilden.
</p>

In [34]:
import json

import basyx.aas.adapter.json
import basyx.aas.adapter.xml

In [35]:
aashell_json_string = json.dumps(aas, cls=basyx.aas.adapter.json.AASToJsonEncoder)

property_json_string = json.dumps(submodel.submodel_element.get_referable('ExampleProperty'),
                                  cls=basyx.aas.adapter.json.AASToJsonEncoder)

# Mit dieser Technik können wir auch Python-Dict- und List-Datenstrukturen mit verschachtelten AAS-Objekten serialisieren:
json_string = json.dumps({'the_submodel': submodel,
                          'the_aas': aas
                          },
                         cls=basyx.aas.adapter.json.AASToJsonEncoder)

print("Abbildung der AAS:\n")
print(aashell_json_string)
print("\n\nAbbildung der ExampleProperty:\n")
print(property_json_string)
print("\n\nAbbildung des gesamten Submodels und der AAS:\n")
print(json_string)
print()


Abbildung der AAS:

{"idShort": "", "modelType": {"name": "AssetAdministrationShell"}, "identification": {"id": "https://acplt.org/Simple_AAS", "idType": "IRI"}, "asset": {"keys": [{"type": "Asset", "idType": "IRI", "value": "https://acplt.org/Simple_Asset", "local": true}]}, "submodels": [{"keys": [{"type": "Submodel", "idType": "IRI", "value": "https://acplt.org/Simple_Submodel", "local": true}]}]}


Abbildung der ExampleProperty:

{"idShort": "ExampleProperty", "modelType": {"name": "Property"}, "semanticId": {"keys": [{"type": "GlobalReference", "idType": "IRI", "value": "http://acplt.org/Properties/SimpleProperty", "local": false}]}, "value": "exampleValue", "valueType": "string"}


Abbildung des gesamten Submodels und der AAS:

{"the_submodel": {"idShort": "", "modelType": {"name": "Submodel"}, "identification": {"id": "https://acplt.org/Simple_Submodel", "idType": "IRI"}, "submodelElements": [{"idShort": "ExampleProperty", "modelType": {"name": "Property"}, "semanticId": {"keys"

### Anlegen von JSON oder XML-Dateien zur Weitergabe

In [36]:
# Schritt 1: Erstellen eines ObjectStore, der die zu serialisierenden Objekte enthält.
# Weitere Informationen finden Sie in `tutorial_storage.py`.
obj_store: model.DictObjectStore[model.Identifiable] = model.DictObjectStore()
obj_store.add(asset)
obj_store.add(submodel)
obj_store.add(aas)

# Schritt 2: Vergewissern Sie sich erneut, dass die Daten auf dem neuesten Stand sind.
asset.update()
submodel.update()
aas.update()

# Schritt 3: Schreiben des Inhalts des ObjectStore in eine JSON-Datei
# Achtung! Es ist wichtig, die Datei im Textmodus mit utf-8-Kodierung zu öffnen!
with open('data.json', 'w', encoding='utf-8') as json_file:
    basyx.aas.adapter.json.write_aas_json_file(json_file, obj_store, indent=4)

# Wir können das zusätzliche Schlüsselwortargument `indent=4` an `write_aas_json_file()` übergeben, 
# um die JSON-Datei auf besser lesbar zu formatieren (aber viel mehr Platz zu verbrauchen).

# Schritt 4: Schreiben des Inhalts des ObjectStore in eine XML-Datei
# Achtung! Für das Schreiben von XML-Dateien -- im Gegensatz zum Schreiben von JSON -- 
# muss die Datei im Binärmodus geöffnet werden! Der XML
# Writer kümmert sich intern um die Zeichenkodierung.
with open('data.xml', 'wb') as xml_file:
    basyx.aas.adapter.xml.write_aas_xml_file(xml_file, obj_store)

### Parsen von einzelnen Objekten oder benutzerdefinierten Datenstrukturen aus JSON- oder XML-Dateien

In [ ]:
# Schritt 1: Lesen des Inhalts der JSON-Datei als ObjectStore
# Achtung! Es ist wichtig, die Datei im Textmodus mit utf-8-Kodierung zu öffnen! Die Verwendung von 'utf-8-sig' wird empfohlen, um
# Unicode Byte Order Marks (BOM) korrekt zu behandeln.
with open('data.json', encoding='utf-8-sig') as json_file:
    json_file_data = basyx.aas.adapter.json.read_aas_json_file(json_file)

# Indem wir das Argument `failsafe=False` an `read_aas_json_file()` übergeben, können wir zum `StrictAASFromJsonDecoder` wechseln

# Schritt 2: Lesen des Inhalts der XML-Datei als ObjectStore
# Achtung! Zum Lesen von XML-Dateien -- im Gegensatz zum Lesen von JSON -- muss die Datei im Binärmodus geöffnet werden! Der XML
# Writer kümmert sich intern um die Zeichenkodierung.
with open('data.xml', 'rb') as xml_file:
    xml_file_data = basyx.aas.adapter.xml.read_aas_xml_file(xml_file)

# Auch hier können wir `failsafe=False` verwenden, um eine strengere Fehlermeldung im Parser einzuschalten.

# Schritt 5.3: Abrufen der Objekte aus dem ObjectStore
# Weitere Informationen über die verfügbaren Techniken finden Sie in `tutorial_storage.py`.
submodel_from_xml = xml_file_data.get_identifiable(model.Identifier('https://acplt.org/Simple_Submodel',
                                                                    model.IdentifierType.IRI))
assert isinstance(submodel_from_xml, model.Submodel)

## Auslesen von assx-Dateien

Am Beispiel einer ASS von Firma Stahl soll gezeigt werden, wie der Datenaustausch und die Weiterverarbeitung funktioniert.
Die Datei kann hier heruntergeladen werden:
https://demo-digital-twin.r-stahl.com/t/205282?lang=de

In [9]:
from basyx.aas.adapter import aasx

objects = model.DictObjectStore()
files = aasx.DictSupplementaryFileContainer()
with aasx.AASXReader("RStahl_205282_Type.aasx") as reader:
    meta_data = reader.get_core_properties()
    reader.read_into(objects, files) 

### Zunächst einfach als JSON ausgeben, das ist ja inzwischen ein bekanntes Format  

In [6]:
with open('RStahl_205282_Type.json', 'w', encoding='utf-8') as json_file:
    basyx.aas.adapter.json.write_aas_json_file(json_file, objects, indent=4)   

### Nun können wir auf die einzelnen Objekte zugreifen:

In [28]:
identifier = model.Identifier(id_='https://demo-digital-twin.r-stahl.com/ids/asset/205282',
                              id_type=model.IdentifierType.IRI)
asset = objects.get(identifier)


identifier = model.Identifier(id_='https://demo-digital-twin.r-stahl.com/t205282',
                              id_type=model.IdentifierType.IRI)
aas = objects.get(identifier)


identifier = model.Identifier(id_='https://demo-digital-twin.r-stahl.com/ids/sm/nameplate/205282',
                              id_type=model.IdentifierType.IRI)
submodel = objects.get(identifier)

print(submodel)

Submodel[Identifier(IRI=https://demo-digital-twin.r-stahl.com/ids/sm/nameplate/205282)]
